![Pytorch](../../../pytorch_logo_2018.svg)

# Pytorch 中级篇（3）：循环神经网络（Recurrent Neural Network）

>参考代码
>
>**yunjey的 [pytorch tutorial系列](https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/recurrent_neural_network/main.py)**

## 循环神经网络 学习资源

一直以为，循环神经网络使用在语音处理上的，跟我这个研究计算机视觉的没有多大关系，所以一直都回避RNN。

这里居然有RNN对MINIST数据的网络实现，那就顺带把RNN给学了。

RNN结合CNN可以用于描述照片，正好能跟计算机视觉结合起来。


>**介绍视频（没有原理）**
>
>[什么是循环神经网络 RNN (深度学习)? What is Recurrent Neural Networks (deep learning)?](https://www.youtube.com/watch?v=EEtf4kNsk7Q)

>**相关网页**
>
>[(新手向)能否简单易懂的介绍一下RNN(循环神经网络)？](https://www.zhihu.com/question/68552209)
>
>[一文搞懂RNN（循环神经网络）基础篇](https://zhuanlan.zhihu.com/p/30844905).
>
>[【译】 理解 LSTM 网络](https://www.jianshu.com/p/9dc9f41f0b29)

当然这些都是基础版本的RNN，RNN的魅力在于它的各种变化版本能用来解决各种不同形式的问题。

[【图片来源】循环神经网络RNN打开手册](https://zhuanlan.zhihu.com/p/22930328)
![RNN的形式](https://pic1.zhimg.com/80/v2-6522f0e0cd9740f45e1ee46591898081_hd.jpg)


## Pytorch实现

**many to one 的形式解决MINIST数据集 手写数字分类问题。**

In [3]:
# 包
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [4]:
# 设备配置
# Device configuration
torch.cuda.set_device(1) # 这句用来设置pytorch在哪块GPU上运行
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# 超参数设置
# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

### MINIST 数据集

In [7]:
# 训练数据
train_dataset = torchvision.datasets.MNIST(root='../../../data/minist/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

# 测试数据
test_dataset = torchvision.datasets.MNIST(root='../../../data/minist/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# 训练数据加载器
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

# 测试数据加载器
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

### 循环神经网络搭建

In [9]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True) # 选用LSTM RNN结构
        self.fc = nn.Linear(hidden_size, num_classes) # 最后一层为全连接层，将隐状态转为分类
    
    def forward(self, x):
        # 初始化隐层状态和细胞状态
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # 前向传播LSTM
        out, _ = self.lstm(x, (h0, c0))  # 输出大小 (batch_size, seq_length, hidden_size)
        
        # 解码最后一个时刻的隐状态
        out = self.fc(out[:, -1, :])
        return out

In [10]:
# 实例化一个模型
# 注意输入维度，虽然我不懂将一幅图28x28拆成28个大小为28的序列有啥意义
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# 定义损失函数和优化器
# Adam: A Method for Stochastic Optimization
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### 训练模型

In [11]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device) # 注意维度
        labels = labels.to(device)
        
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化，注意梯度每次清零
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/600], Loss: 0.4569
Epoch [1/2], Step [200/600], Loss: 0.2823
Epoch [1/2], Step [300/600], Loss: 0.3512
Epoch [1/2], Step [400/600], Loss: 0.1702
Epoch [1/2], Step [500/600], Loss: 0.3181
Epoch [1/2], Step [600/600], Loss: 0.1821
Epoch [2/2], Step [100/600], Loss: 0.1540
Epoch [2/2], Step [200/600], Loss: 0.0848
Epoch [2/2], Step [300/600], Loss: 0.1985
Epoch [2/2], Step [400/600], Loss: 0.1537
Epoch [2/2], Step [500/600], Loss: 0.0988
Epoch [2/2], Step [600/600], Loss: 0.0315


### 测试模型并保存

In [14]:
# 测试集
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

Test Accuracy of the model on the 10000 test images: 97.47 %


In [15]:
# 保存模型
torch.save(model.state_dict(), 'model.ckpt')